<a href="https://colab.research.google.com/github/DaAnMaGi/PI_ML_OPS/blob/main/formulacion/formulacion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descarga de bibliotecas a utilizar

In [4]:
# Se descargan las bibliotecas necesarias
!pip install nltk
# !pip install googletrans
!pip install unidecode
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=f60bf9681295429f0cb6d05f470ce466a8dc455e4a7a1c80032a025ee25f6743
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [22]:
# Se importan las bibliotecas a utilizar

# Pandas y numpy
import pandas as pd
import numpy as np

# Trabajo de archivos
import gzip
import json
from pandas import json_normalize

# Detección de idioma
from googletrans import Translator
translator = Translator()
from langdetect import detect, LangDetectException

#
import itertools

# Gráficación
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Análisis de lenguaje natural
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Se cargan las bases

In [20]:
# Se traen las rutas de los archivos a trabajar.
games_ruta = "/content/games_clean.json.gz"
genres_ruta = "/content/genres_games.json.gz"
specs_ruta = "/content/specs_games.json.gz"
reviews_ruta = "/content/reviews_limpia.json.gz"
items_ruta = "/content/items_limpia.json.gz"

In [137]:
# Se convierten a pandas.

In [8]:
games = pd.read_json(games_ruta,compression="gzip")
games.head()

,name,release_date,id_game,developer
0,Lost Summoner Kitty,2018-01-04,761140,Kotoshiro
1,Ironbound,2018-01-04,643980,Secret Level SRL
2,Real Pool 3D - Poolians,2017-07-24,670290,Poolians.com
3,弹炸人2222,2017-12-07,767400,彼岸领域
4,Log Challenge,None,773570,None


In [9]:
genres = pd.read_json(genres_ruta,compression="gzip")
genres.head()

,id_game,genre
0,761140,Strategy
1,761140,Action
2,761140,Indie
3,761140,Casual
4,761140,Simulation


In [10]:
specs = pd.read_json(specs_ruta,compression="gzip")
# specs["id_game"] = specs["id_game"].astype(int)
specs.head()

,id_game,specs
0,761140,Single-player
1,643980,Single-player
2,643980,Multi-player
3,643980,Online Multi-Player
4,643980,Cross-Platform Multiplayer


In [11]:
reviews = pd.read_json(reviews_ruta,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews.head()

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08


In [21]:
items = pd.read_json(items_ruta,compression="gzip")
items

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0
...,...,...,...,...,...
5159890,76561198329548331,346330,BrainBread 2,0,0
5159891,76561198329548331,373330,All Is Dust,0,0
5159892,76561198329548331,388490,One Way To Die: Steam Edition,3,3
5159893,76561198329548331,521570,You Have 10 Seconds 2,4,4


# Procesamiento del lenguaje natural en las reviews.

In [12]:
reviews

,user_id,item_id,recommend,review,review_date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...,...,...,...
43971,wayfeng,730,True,its FUNNNNNNNN,2015-10-14
43972,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015-10-10
43973,72947282842,730,True,Prettyy Mad Game,2015-10-31
43974,ApxLGhost,730,True,AMAZING GAME 10/10,2015-12-14


In [13]:
# Se obtiene información sobre la distribución del dataset.
reviews["recommend"].value_counts(normalize=True)

True     0.906972
False    0.093028
Name: recommend, dtype: float64

In [14]:
# Se transliteran los caracteres de "review" a ASCII
reviews["review"] = reviews["review"].apply(unidecode)
# Se colocan todos los caracteres en minúscula y se asegura que sean strings.
reviews["review"] = reviews["review"].str.lower()

In [15]:
# Se crea una función para limpiar el dataset de números y caracteres no válidos.
def limpieza(texto):
  return re.sub("[^a-zA-Zñéóúíá]"," ",texto)

# Se hace limpieza del dataset.
reviews["review"] = reviews["review"].apply(limpieza)

In [16]:
# Se crea una función para detectar el lenguaje (ya que aplicar limpieza de manera directa no estaba dando resultado)
# En caso de que no sea capaz de detectarlo, no trae ningún valor.
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        return None

In [17]:
# Se detecta el lenguaje para cada review
reviews["language"] = reviews["review"].apply(detect_language)

In [18]:
# Se revisa la base obtenida.
reviews

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it s unique and worth a playthrough,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple it s actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43971,wayfeng,730,True,its funnnnnnnn,2015-10-14,no
43972,76561198251004808,253980,True,awesome fantasy game if you don t mind the gra...,2015-10-10,en
43973,72947282842,730,True,prettyy mad game,2015-10-31,no
43974,ApxLGhost,730,True,amazing game,2015-12-14,tl


In [23]:
# Se decide guardar la base con los idiomas detectados.

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

guardar_archivo(reviews,"/content/reviews_lang.json.gz")


In [25]:
# Se decide seguir trabajando con la nueva base de reviews a partir del nuevo archivo
ruta_reviews_lang = "/content/reviews_lang.json.gz"
reviews = pd.read_json(ruta_reviews_lang,compression="gzip",encoding="utf-8",convert_dates=['review_date'],date_unit="ms")
reviews

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it s unique and worth a playthrough,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple it s actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43971,wayfeng,730,True,its funnnnnnnn,2015-10-14,no
43972,76561198251004808,253980,True,awesome fantasy game if you don t mind the gra...,2015-10-10,en
43973,72947282842,730,True,prettyy mad game,2015-10-31,no
43974,ApxLGhost,730,True,amazing game,2015-12-14,tl


In [26]:
# Se revisan los idiomas obtenidos a partir de la detección.
reviews["language"].unique()

# Es importante notar que hay reviews (por ejemplo, del inglés y del español)
# que fueron detectadas en otros idiomas, esto debido a errores de escritura
# u ortografía.

array(['en', 'tl', 'so', 'sv', 'af', 'cy', 'pl', 'pt', 'sl', 'fi', 'hr',
       'id', 'lt', 'hu', 'da', 'it', 'no', 'sw', 'nl', 'ro', 'es', None,
       'cs', 'fr', 'ca', 'de', 'et', 'tr', 'sq', 'sk', 'lv', 'vi'],
      dtype=object)

In [27]:
# Se obtiene la información del dataframe por lenguaje.
reviews.groupby(["language"],dropna=False).count()

,user_id,item_id,recommend,review,review_date
language,,,,,
af,870,870,870,870,870
ca,266,266,266,266,266
cs,44,44,44,44,44
cy,527,527,527,527,527
da,634,634,634,634,634
de,102,102,102,102,102
en,32983,32983,32983,32983,32983
es,1072,1072,1072,1072,1072
et,363,363,363,363,363


In [28]:
# Se escogen las reviews detectadas correctamente del idioma inglés ("en")
reviews_en = reviews[reviews["language"] == "en"]
reviews_en

,user_id,item_id,recommend,review,review_date,language
0,76561197970982479,1250,True,simple yet with great replayability in my opi...,2011-11-05,en
1,76561197970982479,22200,True,it s unique and worth a playthrough,2011-07-15,en
2,76561197970982479,43110,True,great atmosphere the gunplay can be a bit chu...,2011-04-21,en
3,js41637,251610,True,i know what you think when you see this title ...,2014-06-24,en
4,js41637,227300,True,for a simple it s actually not all that simpl...,2013-09-08,en
...,...,...,...,...,...,...
43967,76561198236893796,209000,True,this game is great this game is very unusual t...,2015-10-21,en
43968,pigeonie,440,True,a pretty fun game i got addicted pretty quick,2015-09-04,en
43969,76561198237650078,730,True,the only game i need,2015-11-28,en
43972,76561198251004808,253980,True,awesome fantasy game if you don t mind the gra...,2015-10-10,en


In [29]:
# Se obtiene información sobre la distribución del dataset.
reviews_en["recommend"].value_counts(normalize=True)

True     0.899797
False    0.100203
Name: recommend, dtype: float64